In [ ]:
import os
from copy import deepcopy
from tqdm import tqdm
from datetime import date, datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, root_mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import optuna
from optuna.samplers import RandomSampler
import warnings
warnings.filterwarnings("ignore")
import multiprocessing
max_n_jobs = multiprocessing.cpu_count()
print(f"Maximum n_jobs you can use: {max_n_jobs}")

In [ ]:
df = pd.read_csv("NMXLNT_df.csv")
df["datetime"] = pd.to_datetime(df["datetime"])
print(df["Location"].unique())
df.head()

In [ ]:
# try to plot cod against time in all data
fig, ax = plt.subplots(nrows = 3, ncols = 2, figsize = (10, 15))
for inx, loc in enumerate(df["Location"].unique()):
    x, y = inx // 2, inx % 2
    sns.scatterplot(df[df["Location"] == loc], x = "datetime", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(loc)
plt.show()

Make CV split & compare with baseline

In [ ]:
def create_cv_split(df, features_used, cv = 5):
    X_train_arr = []
    X_test_arr = []
    Y_train_arr = []
    Y_test_arr = []
    start_month = 13 - cv
    for i in range(cv):
        train = deepcopy(df[df["datetime"].dt.month < start_month + i].reset_index().drop("index", axis = 1))
        test = deepcopy(df[(df["datetime"].dt.month >= start_month + i) & (df["datetime"].dt.month < start_month + 1 + i)].reset_index().drop("index", axis = 1))
        #print(train.shape[0] / (train.shape[0] + test.shape[0]))
        X_train_arr.append(train[features_used])
        X_test_arr.append(test[features_used])
        Y_train_arr.append(train["cod"])
        Y_test_arr.append(test["cod"])
    return X_train_arr, X_test_arr, Y_train_arr, Y_test_arr

def create_cv_split_with_val(df, features_used, cv = 5):
    X_train_arr = []
    X_val_arr = []
    X_test_arr = []
    Y_train_arr = []
    Y_val_arr = []
    Y_test_arr = []
    start_month = 13 - cv
    for i in range(cv):
        train = deepcopy(df[df["datetime"].dt.month < start_month + i - 1].reset_index().drop("index", axis = 1))
        val = deepcopy(df[df["datetime"].dt.month == start_month + i - 1].reset_index().drop("index", axis = 1))
        test = deepcopy(df[(df["datetime"].dt.month >= start_month + i) & (df["datetime"].dt.month < start_month + 1 + i)].reset_index().drop("index", axis = 1))
        #print(train.shape[0] / (train.shape[0] + test.shape[0]))
        X_train_arr.append(train[features_used])
        X_val_arr.append(val[features_used])
        X_test_arr.append(test[features_used])
        Y_train_arr.append(train["cod"])
        Y_val_arr.append(val["cod"])
        Y_test_arr.append(test["cod"])
    return X_train_arr, X_val_arr, X_test_arr, Y_train_arr, Y_val_arr, Y_test_arr

def create_cv_split_location(df, features_used, loc, cv = 5):
    df = df[df["Location"] == loc].reset_index().drop("index", axis = 1)
    X_train_arr = []
    X_test_arr = []
    Y_train_arr = []
    Y_test_arr = []
    start_month = 13 - cv
    for i in range(cv):
        train = deepcopy(df[df["datetime"].dt.month < start_month + i].reset_index().drop("index", axis = 1))
        test = deepcopy(df[(df["datetime"].dt.month >= start_month + i) & (df["datetime"].dt.month < start_month + 1 + i)].reset_index().drop("index", axis = 1))
        #print(train.shape[0] / test.shape[0])
        X_train_arr.append(train[features_used])
        X_test_arr.append(test[features_used])
        Y_train_arr.append(train["cod"])
        Y_test_arr.append(test["cod"])
    return X_train_arr, X_test_arr, Y_train_arr, Y_test_arr

def create_cv_split_diff(df, features_used, time_diff = 4, cv = 5):
    X_train_arr = []
    X_test_arr = []
    Y_train_arr = []
    Y_test_arr = []
    start_month = 13 - cv
    for i in range(cv):
        train = deepcopy(df[df["datetime"].dt.month < start_month + i].reset_index().drop("index", axis = 1))
        test = deepcopy(df[(df["datetime"].dt.month >= start_month + i) & (df["datetime"].dt.month < start_month + 1 + i)].reset_index().drop("index", axis = 1))
        #print(train.shape[0] / test.shape[0])
        X_train_arr.append(train[features_used])
        X_test_arr.append(test[features_used])
        Y_train_arr.append(train[f"cod_diff_{time_diff}"])
        Y_test_arr.append(test[f"cod_diff_{time_diff}"])
    return X_train_arr, X_test_arr, Y_train_arr, Y_test_arr

def create_cv_split_with_info(df, features_used, cv = 5):
    X_train_arr = []
    X_test_arr = []
    Y_train_arr = []
    Y_test_arr = []
    info_train_arr = []
    info_test_arr = []
    start_month = 13 - cv
    for i in range(cv):
        train = deepcopy(df[df["datetime"].dt.month < start_month + i].reset_index().drop("index", axis = 1))
        test = deepcopy(df[(df["datetime"].dt.month >= start_month + i) & (df["datetime"].dt.month < start_month + 1 + i)].reset_index().drop("index", axis = 1))
        #print(train.shape[0] / test.shape[0])
        X_train_arr.append(train[features_used])
        X_test_arr.append(test[features_used])
        Y_train_arr.append(train["cod"])
        Y_test_arr.append(test["cod"])
        info_train_arr.append(train[["datetime", "Location"]])
        info_test_arr.append((test[["datetime", "Location"]]))
    return X_train_arr, X_test_arr, Y_train_arr, Y_test_arr, info_train_arr, info_test_arr

Make the model and finetune

In [ ]:
# Finetuning XGBoost
def objective_xgboost(trial):
    params = {
        "n_estimators": 200,
        "max_depth": trial.suggest_int("max_depth", 1, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, log = True),
        "verbosity": 0,
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0, 1),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "enable_categorical": True,
        "random_state": 101
    }

    xgbr = XGBRegressor(**params)
    cv_rmse = 0

    for i in range(5):
        X_train, X_test = X_train_arr[i], X_test_arr[i]
        Y_train, Y_test = Y_train_arr[i], Y_test_arr[i]
        xgbr.fit(X_train, Y_train)
        Y_pred = xgbr.predict(X_test)
        cv_rmse += root_mean_squared_error(Y_test, Y_pred)
    
    return cv_rmse / 5

def objective_lightgbm(trial):
    params = {
        "n_estimators": 200,
        "verbosity": -1,
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 2**10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
        "random_state": 101
    }

    lgbr = LGBMRegressor(**params)
    cv_rmse = 0

    for i in range(5):
        X_train, X_test = X_train_arr[i], X_test_arr[i]
        Y_train, Y_test = Y_train_arr[i], Y_test_arr[i]
        lgbr.fit(X_train, Y_train)
        Y_pred = lgbr.predict(X_test)
        cv_rmse += root_mean_squared_error(Y_test, Y_pred)
    
    return cv_rmse / 5

def objective_catboost(trial):
    params = {
        "iterations": 200,
        "verbose": False,
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 2, 600),
        "random_seed": 37
    }

    cbr = CatBoostRegressor(**params)
    cv_rmse = 0

    for i in range(5):
        X_train, X_test = X_train_arr[i], X_test_arr[i]
        Y_train, Y_test = Y_train_arr[i], Y_test_arr[i]
        cbr.fit(X_train, Y_train)
        Y_pred = cbr.predict(X_test)
        cv_rmse += root_mean_squared_error(Y_test, Y_pred)
    
    return cv_rmse / 5

def objective_xgboost_with_val(trial):
    params = {
        "n_estimators": 1000,
        "max_depth": trial.suggest_int("max_depth", 1, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, log = True),
        "verbosity": 0,
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0, 1),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "enable_categorical": True,
        "random_state": 101,
        "early_stopping_rounds": 20
    }

    xgbr = XGBRegressor(**params)
    cv_rmse = 0

    for i in range(5):
        X_train, X_val, X_test = X_train_arr[i], X_val_arr[i], X_test_arr[i]
        Y_train, Y_val, Y_test = Y_train_arr[i], Y_val_arr[i], Y_test_arr[i]
        xgbr.fit(
            X_train, 
            Y_train,
            eval_set=[(X_val, Y_val)],
            verbose=False
        )
        Y_pred = xgbr.predict(X_test)
        cv_rmse += root_mean_squared_error(Y_test, Y_pred)
    
    return cv_rmse / 5

def objective_catboost_with_val(trial):
    params = {
        "iterations": 1000,
        "verbose": False,
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 2, 600),
        "random_seed": 37,
        "early_stopping_rounds": 20
    }

    cbr = CatBoostRegressor(**params)
    cv_rmse = 0

    for i in range(5):
        X_train, X_val, X_test = X_train_arr[i], X_val_arr[i], X_test_arr[i]
        Y_train, Y_val, Y_test = Y_train_arr[i], Y_val_arr[i], Y_test_arr[i]
        cbr.fit(
            X_train, 
            Y_train,
            eval_set=[(X_val, Y_val)],
            verbose=False
        )
        Y_pred = cbr.predict(X_test)
        cv_rmse += root_mean_squared_error(Y_test, Y_pred)
    
    return cv_rmse / 5

In [ ]:
def optimize_xgboost(study_name, storage_name, objective_function=objective_xgboost, n_trials = 50):
    print("Conduct hyperparam opt for XGBoost")
    study = optuna.create_study(
        study_name = study_name,
        direction ='minimize',
        storage = f"sqlite:///{storage_name}.db",
        sampler = RandomSampler(seed = 101),
        load_if_exists=True
    )
    study.optimize(objective_function, n_trials=n_trials, n_jobs=2)
    print('Best hyperparameters:', study.best_params)
    print('Best RMSE:', study.best_value)
    return study.best_params

def optimize_lightgbm(study_name, storage_name, objective_function=objective_lightgbm, n_trials = 50):
    print("Conduct hyperparam opt for LightGBM")
    study = optuna.create_study(
        study_name = study_name,
        direction='minimize',
        storage = f"sqlite:///{storage_name}.db",
        sampler = RandomSampler(seed = 101),
        load_if_exists=True
    )
    study.optimize(objective_function, n_trials=n_trials, n_jobs=2)
    print('Best hyperparameters:', study.best_params)
    print('Best MSE:', study.best_value)
    return study.best_params

def optimize_catboost(study_name, storage_name, objective_function=objective_catboost, n_trials = 50):
    print("Conduct hyperparam opt for CatBoost")
    study = optuna.create_study(
        study_name = study_name,
        direction='minimize',
        storage = f"sqlite:///{storage_name}.db",
        sampler = RandomSampler(seed = 101),
        load_if_exists=True
    )
    study.optimize(objective_function, n_trials=n_trials, n_jobs=2)
    print('Best hyperparameters:', study.best_params)
    print('Best RMSLE:', study.best_value)
    return study.best_params

In [ ]:
# use 1 month for test and previous montsh for predict
# Take 1: only use features appear in all data
features_used = [f"cod_prev_{i}" for i in range(4, 13)] + \
                [f"temp_prev_{i}" for i in range(4, 13)] + \
                [f"ph_prev_{i}" for i in range(4, 13)] + \
                [f"tss_prev_{i}" for i in range(4, 13)] + \
                ["sin_hour", "sin_day", "sin_month"] # try to use sine version instead of numerical due to the cyclical nature of date
#                [f"nh4_prev_{i}" for i in range(4, 9)]
# features_used = [c for c in df.columns if "prev" in c and df[c].dtypes in ["int64", "float64"]]
X_train_arr, X_test_arr, Y_train_arr, Y_test_arr = create_cv_split(df, features_used)

# baseline: using last k hours to predict
baseline_score = 0
for i in range(5):
    baseline_score += np.sqrt(np.mean((Y_test_arr[i] - X_test_arr[i][f"cod_prev_4"])**2))
baseline_score /= 5
print(baseline_score)

for loc in df["Location"].unique():
    print(loc)
    temp_df = deepcopy(df[df["Location"] == loc])
    X_train_arr, X_test_arr, Y_train_arr, Y_test_arr = create_cv_split(temp_df, features_used)
    for i in range(5):
        print(i, np.sqrt(np.mean((Y_test_arr[i] - X_test_arr[i][f"cod_prev_4"])**2)))

X_train_arr, X_test_arr, Y_train_arr, Y_test_arr = create_cv_split(df, features_used)

In [ ]:
best_params_xgboost = optimize_xgboost(
    f"xgboost_study_{str(date.today())}", 
    f"xgboost_study_{str(date.today())}"
)
# best is 2.85

In [ ]:
best_params_lightgbm = optimize_lightgbm(
    f"lightgbm_study_{str(date.today())}", 
    f"lightgbm_study_{str(date.today())}"
) 
# best is 2.96

In [ ]:
best_params_catboost = optimize_catboost(
    f"catboost_study_{str(date.today())}", 
    f"catboost_study_{str(date.today())}"
)
# best is this one, about 2.81 RMSE

Testing on best configuration

In [ ]:
params = {
    "n_estimators": 200,
    "verbosity": 0,
    "enable_categorical": True,
    "random_state": 101
}
for p in best_params_xgboost:
    params[p] = best_params_xgboost[p]

feature_importances = {}

xgbr = XGBRegressor(**params)
for i in range(5):
    X_train, X_test = X_train_arr[i], X_test_arr[i]
    Y_train, Y_test = Y_train_arr[i], Y_test_arr[i]
    xgbr.fit(X_train, Y_train)
    features = xgbr.feature_names_in_.tolist()
    features_i = xgbr.feature_importances_.tolist()
    for inx, feat in enumerate(features):
        feature_importances[feat] = feature_importances.get(feat, 0) + features_i[inx]

feature_importances
# Seems like only COD features are important (can try to only use 4-8 hours if 4-13 hours does not work well)

In [ ]:
params = {
    "n_estimators": 200,
    "verbosity": -1,
    "random_state": 101,
}
for p in best_params_lightgbm:
    params[p] = best_params_lightgbm[p]

feature_importances = {}

lgbr = LGBMRegressor(**params)
for i in range(5):
    X_train = X_train_arr[i]
    Y_train = Y_train_arr[i]
    lgbr.fit(X_train, Y_train)
    features = lgbr.feature_names_in_.tolist()
    features_i = lgbr.feature_importances_.tolist()
    for inx, feat in enumerate(features):
        feature_importances[feat] = feature_importances.get(feat, 0) + features_i[inx]

feature_importances
# seems to pick up time features not as good as past 4 hours features

In [ ]:
params = {
    "iterations": 200,
    "verbose": False,
    "random_seed": 37
}
for p in best_params_catboost:
    params[p] = best_params_catboost[p]

feature_importances = {}

cbr = CatBoostRegressor(**params)
cv_rmse = 0

for i in range(5):
    X_train = X_train_arr[i]
    Y_train = Y_train_arr[i]
    cbr.fit(X_train, Y_train)
    features = cbr.feature_names_
    features_i = cbr.feature_importances_.tolist()
    for inx, feat in enumerate(features):
        feature_importances[feat] = feature_importances.get(feat, 0) + features_i[inx]

feature_importances
# can pick up a combination of both past cod and tss, not good at picking up ph, temp

Try to train with only previous CODs insteads

In [ ]:
features_used = [f"cod_prev_{i}" for i in range(4, 13)] 
X_train_arr, X_test_arr, Y_train_arr, Y_test_arr = create_cv_split(df, features_used)

In [ ]:
best_params_xgboost_only_cod = optimize_xgboost(
    "xgboost_study_only_cod_2025-05-03", #f"xgboost_study_only_cod_{str(date.today())}",
    "xgboost_study_only_cod_2025-05-03" #f"xgboost_study_only_cod_{str(date.today())}",
)
# seems to not improve?, might be because using all 4-12 hours cod before does not work => use 4-8 hours before
# still 2.87 in the best version that only use cod, worse than best version that use all features
# seems to better by including other features than just time features

In [ ]:
# try to train with less values of cod
features_used = [f"cod_prev_{i}" for i in range(4, 9)] 
X_train_arr, X_test_arr, Y_train_arr, Y_test_arr = create_cv_split(df, features_used)

In [ ]:
best_params_xgboost_only_cod_truncated = optimize_xgboost(
    f"xgboost_study_only_cod_truncated_{str(date.today())}",
    f"xgboost_study_only_cod_truncated_{str(date.today())}",
) #worse

In [ ]:
features_used = [f"cod_prev_{i}" for i in range(4, 13)] 
X_train_arr, X_test_arr, Y_train_arr, Y_test_arr = create_cv_split(df, features_used)

params = {
    "n_estimators": 200,
    "verbosity": 0,
    "enable_categorical": True,
    "random_state": 101
}
for p in best_params_xgboost_only_cod:
    params[p] = best_params_xgboost_only_cod[p]

feature_importances = {}

xgbr = XGBRegressor(**params)
for i in range(5):
    X_train = X_train_arr[i]
    Y_train = Y_train_arr[i]
    xgbr.fit(X_train, Y_train)
    features = xgbr.feature_names_in_.tolist()
    features_i = xgbr.feature_importances_.tolist()
    for inx, feat in enumerate(features):
        feature_importances[feat] = feature_importances.get(feat, 0) + features_i[inx]

feature_importances

In [ ]:
# Interesting relationship between importance of cod_prev_4, 5, 12 vs cod => try to only use them?
features_used = [f"cod_prev_{i}" for i in [4, 5, 6, 12]] 
X_train_arr, X_test_arr, Y_train_arr, Y_test_arr = create_cv_split(df, features_used)

In [ ]:
best_params_xgboost_only_cod_truncated_2 = optimize_xgboost(
    f"xgboost_study_only_cod_truncated_2_{str(date.today())}",
    f"xgboost_study_only_cod_truncated_2_{str(date.today())}",
) # better than truncated, not as good as truncated 2

In [ ]:
# params = {
#     "iterations": 200,
#     "verbose": False,
#     "random_seed": 37
# }
# for p in best_params_catboost_only_cod:
#     params[p] = best_params_catboost_only_cod[p]

# feature_importances = {}

# cbr = CatBoostRegressor(**params)
# cv_rmse = 0

# for i in range(5):
#     X_train = X_train_arr[i]
#     Y_train = Y_train_arr[i]
#     cbr.fit(X_train, Y_train)
#     features = cbr.feature_names_
#     features_i = cbr.feature_importances_.tolist()
#     for inx, feat in enumerate(features):
#         feature_importances[feat] = feature_importances.get(feat, 0) + features_i[inx]

# feature_importances

Try to using only last 4 hours cod and some non-cod features

In [ ]:
# use 1 month for test and previous montsh for predict
# Take 1: only use features appear in all data
features_used = ["cod_prev_4", "temp_prev_4", "ph_prev_4", "tss_prev_4", "hour", "day", "month"]
X_train_arr, X_test_arr, Y_train_arr, Y_test_arr = create_cv_split(df, features_used)
# not really good, try to use an extended version

In [ ]:
best_params_lightgbm_only_4h = optimize_lightgbm(
    f"lightgbm_study_only_4h_{str(date.today())}",
    f"lightgbm_study_only_4h_{str(date.today())}"
)

In [ ]:
params = {
    "n_estimators": 200,
    "verbosity": -1,
    "random_state": 101
}
for p in best_params_lightgbm_only_4h:
    params[p] = best_params_lightgbm_only_4h[p]

feature_importances = {}

lgbr = LGBMRegressor(**params)
for i in range(5):
    X_train = X_train_arr[i]
    Y_train = Y_train_arr[i]
    lgbr.fit(X_train, Y_train)
    features = lgbr.feature_names_in_.tolist()
    features_i = lgbr.feature_importances_.tolist()
    for inx, feat in enumerate(features):
        feature_importances[feat] = feature_importances.get(feat, 0) + features_i[inx]

feature_importances

In [ ]:
# use 1 month for test and previous montsh for predict
# Take 1: only use features appear in all data
features_used = ["cod_prev_4", "ph_prev_4", "tss_prev_4", "month"]
X_train_arr, X_test_arr, Y_train_arr, Y_test_arr = create_cv_split(df, features_used)
# not really good, try to use an extended version

In [ ]:
best_params_lightgbm_only_4h_truncated = optimize_lightgbm(
    f"lightgbm_study_only_4h_truncated_{str(date.today())}",
    f"lightgbm_study_only_4h_truncated_{str(date.today())}"
)
# worse

Try to look at special combination based on catboost

In [ ]:
features_used = [f"cod_prev_{i}" for i in range(4, 13)] + \
                ["hour", "day", "month"] + \
                ["tss_prev_4", "tss_prev_5"]

X_train_arr, X_test_arr, Y_train_arr, Y_test_arr = create_cv_split(df, features_used)

In [ ]:
best_params_catboost_best_comb = optimize_catboost(
    "catboost_study_best_comb_2025-05-03", #f"catboost_study_best_comb_{str(date.today())}",
    "catboost_study_best_comb_2025-05-03" #f"catboost_study_best_comb_{str(date.today())}"
)
#2.77, current best model

In [ ]:
params = {
    "iterations": 200,
    "verbose": False,
    "random_seed": 37
}
for p in best_params_catboost_best_comb:
    params[p] = best_params_catboost_best_comb[p]

feature_importances = {}

cbr = CatBoostRegressor(**params)
cv_rmse = 0

for i in range(5):
    X_train = X_train_arr[i]
    Y_train = Y_train_arr[i]
    cbr.fit(X_train, Y_train)
    features = cbr.feature_names_
    features_i = cbr.feature_importances_.tolist()
    for inx, feat in enumerate(features):
        feature_importances[feat] = feature_importances.get(feat, 0) + features_i[inx]

feature_importances

Try to look at specific region

In [ ]:
baymau_df = df[df["Location"] == "BAY MAU.csv"].reset_index().drop("index", axis = 1)

In [ ]:
features_used = [f"cod_prev_{i}" for i in range(4, 9)]
X_train_arr, X_test_arr, Y_train_arr, Y_test_arr = create_cv_split(baymau_df, features_used)

# baseline
baseline_score = 0
for i in range(5):
    baseline_score += np.sqrt(np.mean((Y_test_arr[i] - X_test_arr[i][f"cod_prev_4"])**2))
baseline_score / 5

In [ ]:
best_params_xgboost_only_cod_baymau = optimize_xgboost(
    f"xgboost_study_only_cod_baymau_{str(date.today())}",
    f"xgboost_study_only_cod_baymau_{str(date.today())}",
)

In [ ]:
btlvt_df = df[df["Location"] == "BTLVT.csv"].reset_index().drop("index", axis = 1)

In [ ]:
features_used = [f"cod_prev_{i}" for i in range(4, 9)]
X_train_arr, X_test_arr, Y_train_arr, Y_test_arr = create_cv_split(btlvt_df, features_used)

# baseline
baseline_score = 0
for i in range(5):
    baseline_score += np.sqrt(np.mean((Y_test_arr[i] - X_test_arr[i][f"cod_prev_4"])**2))
baseline_score / 5

In [ ]:
best_params_xgboost_only_cod_btlvt = optimize_xgboost(
    f"xgboost_study_only_cod_btlvt_{str(date.today())}",
    f"xgboost_study_only_cod_btlvt_{str(date.today())}",
)

In [ ]:
caunga_df = df[df["Location"] == "CAU NGA.csv"].reset_index().drop("index", axis = 1)

In [ ]:
features_used = [f"cod_prev_{i}" for i in range(4, 9)]
X_train_arr, X_test_arr, Y_train_arr, Y_test_arr = create_cv_split(caunga_df, features_used)

# baseline
baseline_score = 0
for i in range(5):
    baseline_score += np.sqrt(np.mean((Y_test_arr[i] - X_test_arr[i][f"cod_prev_4"])**2))
baseline_score / 5

In [ ]:
best_params_xgboost_only_cod_caunga = optimize_xgboost(
    f"xgboost_study_only_cod_caunga_{str(date.today())}",
    f"xgboost_study_only_cod_caunga_{str(date.today())}",
)

Analyze error from model

Best model that only use COD

In [ ]:
features_used = [f"cod_prev_{i}" for i in range(4, 13)] + \
                [f"temp_prev_{i}" for i in range(4, 13)] + \
                [f"ph_prev_{i}" for i in range(4, 13)] + \
                [f"tss_prev_{i}" for i in range(4, 13)] + \
                ["hour", "day", "month"]
X_train_arr, X_test_arr, Y_train_arr, Y_test_arr, info_train_arr, info_test_arr = create_cv_split_with_info(df, features_used)

In [ ]:
params = {
    "iterations": 200,
    "verbose": False,
    "random_seed": 37
}
for p in best_params_catboost_only_cod:
    params[p] = best_params_catboost_only_cod[p]

cbr = CatBoostRegressor(**params)

X_train, X_test = X_train_arr[0], X_test_arr[0]
Y_train, Y_test = Y_train_arr[0], Y_test_arr[0]
info_train, info_test = info_train_arr[0], info_test_arr[0]
cbr.fit(X_train, Y_train)
Y_pred = cbr.predict(X_test)
temp = pd.DataFrame({
    "Location": info_test["Location"],
    "datetime": info_test["datetime"],
    "true_cod": Y_test,
    "pred_cod": Y_pred
})

print(root_mean_squared_error(Y_test, Y_pred))
fig, ax = plt.subplots(nrows = 3, ncols = 2, figsize = (10, 15))
for inx, loc in enumerate(df["Location"].unique()):
    temp_df = temp[temp["Location"] == loc]
    print(loc, root_mean_squared_error(temp_df["true_cod"], temp_df["pred_cod"]))
    x, y = inx // 2, inx % 2
    ax[x][y].scatter(temp_df["datetime"], temp_df["true_cod"], color = "green")
    ax[x][y].scatter(temp_df["datetime"], temp_df["pred_cod"], color = "red")
    ax[x][y].set_title(loc)

In [ ]:
params = {
    "iterations": 200,
    "verbose": False,
    "random_seed": 37
}
for p in best_params_catboost_only_cod:
    params[p] = best_params_catboost_only_cod[p]

cbr = CatBoostRegressor(**params)

X_train, X_test = X_train_arr[1], X_test_arr[1]
Y_train, Y_test = Y_train_arr[1], Y_test_arr[1]
info_train, info_test = info_train_arr[1], info_test_arr[1]
cbr.fit(X_train, Y_train)
Y_pred = cbr.predict(X_test)
temp = pd.DataFrame({
    "Location": info_test["Location"],
    "datetime": info_test["datetime"],
    "true_cod": Y_test,
    "pred_cod": Y_pred
})

print(root_mean_squared_error(Y_test, Y_pred))
fig, ax = plt.subplots(nrows = 3, ncols = 2, figsize = (10, 15))
for inx, loc in enumerate(df["Location"].unique()):
    temp_df = temp[temp["Location"] == loc]
    print(loc, root_mean_squared_error(temp_df["true_cod"], temp_df["pred_cod"]))
    x, y = inx // 2, inx % 2
    ax[x][y].scatter(temp_df["datetime"], temp_df["true_cod"], color = "green")
    ax[x][y].scatter(temp_df["datetime"], temp_df["pred_cod"], color = "red")
    ax[x][y].set_title(loc)

In [ ]:
params = {
    "iterations": 200,
    "verbose": False,
    "random_seed": 37
}
for p in best_params_catboost_only_cod:
    params[p] = best_params_catboost_only_cod[p]

cbr = CatBoostRegressor(**params)

X_train, X_test = X_train_arr[2], X_test_arr[2]
Y_train, Y_test = Y_train_arr[2], Y_test_arr[2]
info_train, info_test = info_train_arr[2], info_test_arr[2]
cbr.fit(X_train, Y_train)
Y_pred = cbr.predict(X_test)
temp = pd.DataFrame({
    "Location": info_test["Location"],
    "datetime": info_test["datetime"],
    "true_cod": Y_test,
    "pred_cod": Y_pred
})

print(root_mean_squared_error(Y_test, Y_pred))
fig, ax = plt.subplots(nrows = 3, ncols = 2, figsize = (10, 15))
for inx, loc in enumerate(df["Location"].unique()):
    temp_df = temp[temp["Location"] == loc]
    print(loc, root_mean_squared_error(temp_df["true_cod"], temp_df["pred_cod"]))
    x, y = inx // 2, inx % 2
    ax[x][y].scatter(temp_df["datetime"], temp_df["true_cod"], color = "green")
    ax[x][y].scatter(temp_df["datetime"], temp_df["pred_cod"], color = "red")
    ax[x][y].set_title(loc)

In [ ]:
params = {
    "iterations": 200,
    "verbose": False,
    "random_seed": 37
}
for p in best_params_catboost_only_cod:
    params[p] = best_params_catboost_only_cod[p]

cbr = CatBoostRegressor(**params)

X_train, X_test = X_train_arr[3], X_test_arr[3]
Y_train, Y_test = Y_train_arr[3], Y_test_arr[3]
info_train, info_test = info_train_arr[3], info_test_arr[3]
cbr.fit(X_train, Y_train)
Y_pred = cbr.predict(X_test)
temp = pd.DataFrame({
    "Location": info_test["Location"],
    "datetime": info_test["datetime"],
    "true_cod": Y_test,
    "pred_cod": Y_pred
})

print(root_mean_squared_error(Y_test, Y_pred))
fig, ax = plt.subplots(nrows = 3, ncols = 2, figsize = (10, 15))
for inx, loc in enumerate(df["Location"].unique()):
    temp_df = temp[temp["Location"] == loc]
    print(loc, root_mean_squared_error(temp_df["true_cod"], temp_df["pred_cod"]))
    x, y = inx // 2, inx % 2
    ax[x][y].scatter(temp_df["datetime"], temp_df["true_cod"], color = "green")
    ax[x][y].scatter(temp_df["datetime"], temp_df["pred_cod"], color = "red")
    ax[x][y].set_title(loc)

In [ ]:
params = {
    "iterations": 200,
    "verbose": False,
    "random_seed": 37
}
for p in best_params_catboost_only_cod:
    params[p] = best_params_catboost_only_cod[p]

cbr = CatBoostRegressor(**params)

X_train, X_test = X_train_arr[4], X_test_arr[4]
Y_train, Y_test = Y_train_arr[4], Y_test_arr[4]
info_train, info_test = info_train_arr[4], info_test_arr[4]
cbr.fit(X_train, Y_train)
Y_pred = cbr.predict(X_test)
temp = pd.DataFrame({
    "Location": info_test["Location"],
    "datetime": info_test["datetime"],
    "true_cod": Y_test,
    "pred_cod": Y_pred
})

print(root_mean_squared_error(Y_test, Y_pred))
fig, ax = plt.subplots(nrows = 3, ncols = 2, figsize = (10, 15))
for inx, loc in enumerate(df["Location"].unique()):
    temp_df = temp[temp["Location"] == loc]
    print(loc, root_mean_squared_error(temp_df["true_cod"], temp_df["pred_cod"]))
    x, y = inx // 2, inx % 2
    ax[x][y].scatter(temp_df["datetime"], temp_df["true_cod"], color = "green")
    ax[x][y].scatter(temp_df["datetime"], temp_df["pred_cod"], color = "red")
    ax[x][y].set_title(loc)

Best model that only use last 4 hours features

In [ ]:
features_used = ["cod_prev_4", "ph_prev_4", "tss_prev_4", "month"]
X_train_arr, X_test_arr, Y_train_arr, Y_test_arr, info_train_arr, info_test_arr = create_cv_split_with_info(df, features_used)

In [ ]:
params = {
    "iterations": 100,
    "verbose": False,
    "random_seed": 37
}
for p in best_params_catboost_only_4h:
    params[p] = best_params_catboost_only_4h[p]

cbr = CatBoostRegressor(**params)

X_train, X_test = X_train_arr[0], X_test_arr[0]
Y_train, Y_test = Y_train_arr[0], Y_test_arr[0]
info_train, info_test = info_train_arr[0], info_test_arr[0]
cbr.fit(X_train, Y_train)
Y_pred = cbr.predict(X_test)
temp = pd.DataFrame({
    "Location": info_test["Location"],
    "datetime": info_test["datetime"],
    "true_cod": Y_test,
    "pred_cod": Y_pred
})

print(root_mean_squared_error(Y_test, Y_pred))
fig, ax = plt.subplots(nrows = 3, ncols = 2, figsize = (10, 15))
for inx, loc in enumerate(df["Location"].unique()):
    temp_df = temp[temp["Location"] == loc]
    x, y = inx // 2, inx % 2
    print(loc, root_mean_squared_error(temp_df["true_cod"], temp_df["pred_cod"]))
    ax[x][y].scatter(temp_df["datetime"], temp_df["true_cod"], color = "green")
    ax[x][y].scatter(temp_df["datetime"], temp_df["pred_cod"], color = "red")
    ax[x][y].set_title(loc)

In [ ]:
params = {
    "iterations": 100,
    "verbose": False,
    "random_seed": 37
}
for p in best_params_catboost_only_4h:
    params[p] = best_params_catboost_only_4h[p]

cbr = CatBoostRegressor(**params)

X_train, X_test = X_train_arr[1], X_test_arr[1]
Y_train, Y_test = Y_train_arr[1], Y_test_arr[1]
info_train, info_test = info_train_arr[1], info_test_arr[1]
cbr.fit(X_train, Y_train)
Y_pred = cbr.predict(X_test)
temp = pd.DataFrame({
    "Location": info_test["Location"],
    "datetime": info_test["datetime"],
    "true_cod": Y_test,
    "pred_cod": Y_pred
})

print(root_mean_squared_error(Y_test, Y_pred))
fig, ax = plt.subplots(nrows = 3, ncols = 2, figsize = (10, 15))
for inx, loc in enumerate(df["Location"].unique()):
    temp_df = temp[temp["Location"] == loc]
    x, y = inx // 2, inx % 2
    print(loc, root_mean_squared_error(temp_df["true_cod"], temp_df["pred_cod"]))
    ax[x][y].scatter(temp_df["datetime"], temp_df["true_cod"], color = "green")
    ax[x][y].scatter(temp_df["datetime"], temp_df["pred_cod"], color = "red")
    ax[x][y].set_title(loc)

In [ ]:
params = {
    "iterations": 100,
    "verbose": False,
    "random_seed": 37
}
for p in best_params_catboost_only_4h:
    params[p] = best_params_catboost_only_4h[p]

cbr = CatBoostRegressor(**params)

X_train, X_test = X_train_arr[2], X_test_arr[2]
Y_train, Y_test = Y_train_arr[2], Y_test_arr[2]
info_train, info_test = info_train_arr[2], info_test_arr[2]
cbr.fit(X_train, Y_train)
Y_pred = cbr.predict(X_test)
temp = pd.DataFrame({
    "Location": info_test["Location"],
    "datetime": info_test["datetime"],
    "true_cod": Y_test,
    "pred_cod": Y_pred
})

print(root_mean_squared_error(Y_test, Y_pred))
fig, ax = plt.subplots(nrows = 3, ncols = 2, figsize = (10, 15))
for inx, loc in enumerate(df["Location"].unique()):
    temp_df = temp[temp["Location"] == loc]
    x, y = inx // 2, inx % 2
    print(loc, root_mean_squared_error(temp_df["true_cod"], temp_df["pred_cod"]))
    ax[x][y].scatter(temp_df["datetime"], temp_df["true_cod"], color = "green")
    ax[x][y].scatter(temp_df["datetime"], temp_df["pred_cod"], color = "red")
    ax[x][y].set_title(loc)

In [ ]:
params = {
    "iterations": 100,
    "verbose": False,
    "random_seed": 37
}
for p in best_params_catboost_only_4h:
    params[p] = best_params_catboost_only_4h[p]

cbr = CatBoostRegressor(**params)

X_train, X_test = X_train_arr[3], X_test_arr[3]
Y_train, Y_test = Y_train_arr[3], Y_test_arr[3]
info_train, info_test = info_train_arr[3], info_test_arr[3]
cbr.fit(X_train, Y_train)
Y_pred = cbr.predict(X_test)
temp = pd.DataFrame({
    "Location": info_test["Location"],
    "datetime": info_test["datetime"],
    "true_cod": Y_test,
    "pred_cod": Y_pred
})

print(root_mean_squared_error(Y_test, Y_pred))
fig, ax = plt.subplots(nrows = 3, ncols = 2, figsize = (10, 15))
for inx, loc in enumerate(df["Location"].unique()):
    temp_df = temp[temp["Location"] == loc]
    x, y = inx // 2, inx % 2
    print(loc, root_mean_squared_error(temp_df["true_cod"], temp_df["pred_cod"]))
    ax[x][y].scatter(temp_df["datetime"], temp_df["true_cod"], color = "green")
    ax[x][y].scatter(temp_df["datetime"], temp_df["pred_cod"], color = "red")
    ax[x][y].set_title(loc)

In [ ]:
params = {
    "iterations": 100,
    "verbose": False,
    "random_seed": 37
}
for p in best_params_catboost_only_4h:
    params[p] = best_params_catboost_only_4h[p]

cbr = CatBoostRegressor(**params)

X_train, X_test = X_train_arr[4], X_test_arr[4]
Y_train, Y_test = Y_train_arr[4], Y_test_arr[4]
info_train, info_test = info_train_arr[4], info_test_arr[4]
cbr.fit(X_train, Y_train)
Y_pred = cbr.predict(X_test)
temp = pd.DataFrame({
    "Location": info_test["Location"],
    "datetime": info_test["datetime"],
    "true_cod": Y_test,
    "pred_cod": Y_pred
})

print(root_mean_squared_error(Y_test, Y_pred))
fig, ax = plt.subplots(nrows = 3, ncols = 2, figsize = (10, 15))
for inx, loc in enumerate(df["Location"].unique()):
    temp_df = temp[temp["Location"] == loc]
    x, y = inx // 2, inx % 2
    print(loc, root_mean_squared_error(temp_df["true_cod"], temp_df["pred_cod"]))
    ax[x][y].scatter(temp_df["datetime"], temp_df["true_cod"], color = "green")
    ax[x][y].scatter(temp_df["datetime"], temp_df["pred_cod"], color = "red")
    ax[x][y].set_title(loc)

Best model that use some combination of both

In [ ]:
features_used = [f"cod_prev_{i}" for i in range(4, 13)] + \
                ["hour", "day", "month"] + \
                ["tss_prev_4", "tss_prev_5"]

X_train_arr, X_test_arr, Y_train_arr, Y_test_arr, info_train_arr, info_test_arr = create_cv_split_with_info(df, features_used)

In [ ]:
params = {
    "iterations": 200,
    "verbose": False,
    "random_seed": 37
}
for p in best_params_catboost_special_comb:
    params[p] = best_params_catboost_special_comb[p]

cbr = CatBoostRegressor(**params)

X_train, X_test = X_train_arr[0], X_test_arr[0]
Y_train, Y_test = Y_train_arr[0], Y_test_arr[0]
info_train, info_test = info_train_arr[0], info_test_arr[0]
cbr.fit(X_train, Y_train)
Y_pred = cbr.predict(X_test)
temp = pd.DataFrame({
    "Location": info_test["Location"],
    "datetime": info_test["datetime"],
    "true_cod": Y_test,
    "pred_cod": Y_pred
})

print(root_mean_squared_error(Y_test, Y_pred))
fig, ax = plt.subplots(nrows = 3, ncols = 2, figsize = (10, 15))
for inx, loc in enumerate(df["Location"].unique()):
    temp_df = temp[temp["Location"] == loc]
    x, y = inx // 2, inx % 2
    print(loc, root_mean_squared_error(temp_df["true_cod"], temp_df["pred_cod"]))
    ax[x][y].scatter(temp_df["datetime"], temp_df["true_cod"], color = "green")
    ax[x][y].scatter(temp_df["datetime"], temp_df["pred_cod"], color = "red")
    ax[x][y].set_title(loc)

In [ ]:
params = {
    "iterations": 200,
    "verbose": False,
    "random_seed": 37
}
for p in best_params_catboost_special_comb:
    params[p] = best_params_catboost_special_comb[p]

cbr = CatBoostRegressor(**params)

X_train, X_test = X_train_arr[1], X_test_arr[1]
Y_train, Y_test = Y_train_arr[1], Y_test_arr[1]
info_train, info_test = info_train_arr[1], info_test_arr[1]
cbr.fit(X_train, Y_train)
Y_pred = cbr.predict(X_test)
temp = pd.DataFrame({
    "Location": info_test["Location"],
    "datetime": info_test["datetime"],
    "true_cod": Y_test,
    "pred_cod": Y_pred
})

print(root_mean_squared_error(Y_test, Y_pred))
fig, ax = plt.subplots(nrows = 3, ncols = 2, figsize = (10, 15))
for inx, loc in enumerate(df["Location"].unique()):
    temp_df = temp[temp["Location"] == loc]
    x, y = inx // 2, inx % 2
    print(loc, root_mean_squared_error(temp_df["true_cod"], temp_df["pred_cod"]))
    ax[x][y].scatter(temp_df["datetime"], temp_df["true_cod"], color = "green")
    ax[x][y].scatter(temp_df["datetime"], temp_df["pred_cod"], color = "red")
    ax[x][y].set_title(loc)

In [ ]:
params = {
    "iterations": 200,
    "verbose": False,
    "random_seed": 37
}
for p in best_params_catboost_special_comb:
    params[p] = best_params_catboost_special_comb[p]

cbr = CatBoostRegressor(**params)

X_train, X_test = X_train_arr[2], X_test_arr[2]
Y_train, Y_test = Y_train_arr[2], Y_test_arr[2]
info_train, info_test = info_train_arr[2], info_test_arr[2]
cbr.fit(X_train, Y_train)
Y_pred = cbr.predict(X_test)
temp = pd.DataFrame({
    "Location": info_test["Location"],
    "datetime": info_test["datetime"],
    "true_cod": Y_test,
    "pred_cod": Y_pred
})

print(root_mean_squared_error(Y_test, Y_pred))
fig, ax = plt.subplots(nrows = 3, ncols = 2, figsize = (10, 15))
for inx, loc in enumerate(df["Location"].unique()):
    temp_df = temp[temp["Location"] == loc]
    x, y = inx // 2, inx % 2
    print(loc, root_mean_squared_error(temp_df["true_cod"], temp_df["pred_cod"]))
    ax[x][y].scatter(temp_df["datetime"], temp_df["true_cod"], color = "green")
    ax[x][y].scatter(temp_df["datetime"], temp_df["pred_cod"], color = "red")
    ax[x][y].set_title(loc)

In [ ]:
params = {
    "iterations": 200,
    "verbose": False,
    "random_seed": 37
}
for p in best_params_catboost_special_comb:
    params[p] = best_params_catboost_special_comb[p]

cbr = CatBoostRegressor(**params)

X_train, X_test = X_train_arr[3], X_test_arr[3]
Y_train, Y_test = Y_train_arr[3], Y_test_arr[3]
info_train, info_test = info_train_arr[3], info_test_arr[3]
cbr.fit(X_train, Y_train)
Y_pred = cbr.predict(X_test)
temp = pd.DataFrame({
    "Location": info_test["Location"],
    "datetime": info_test["datetime"],
    "true_cod": Y_test,
    "pred_cod": Y_pred
})

print(root_mean_squared_error(Y_test, Y_pred))
fig, ax = plt.subplots(nrows = 3, ncols = 2, figsize = (10, 15))
for inx, loc in enumerate(df["Location"].unique()):
    temp_df = temp[temp["Location"] == loc]
    x, y = inx // 2, inx % 2
    print(loc, root_mean_squared_error(temp_df["true_cod"], temp_df["pred_cod"]))
    ax[x][y].scatter(temp_df["datetime"], temp_df["true_cod"], color = "green")
    ax[x][y].scatter(temp_df["datetime"], temp_df["pred_cod"], color = "red")
    ax[x][y].set_title(loc)

In [ ]:
params = {
    "iterations": 200,
    "verbose": False,
    "random_seed": 37
}
for p in best_params_catboost_special_comb:
    params[p] = best_params_catboost_special_comb[p]

cbr = CatBoostRegressor(**params)

X_train, X_test = X_train_arr[4], X_test_arr[4]
Y_train, Y_test = Y_train_arr[4], Y_test_arr[4]
info_train, info_test = info_train_arr[4], info_test_arr[4]
cbr.fit(X_train, Y_train)
Y_pred = cbr.predict(X_test)
temp = pd.DataFrame({
    "Location": info_test["Location"],
    "datetime": info_test["datetime"],
    "true_cod": Y_test,
    "pred_cod": Y_pred
})

print(root_mean_squared_error(Y_test, Y_pred))
fig, ax = plt.subplots(nrows = 3, ncols = 2, figsize = (10, 15))
for inx, loc in enumerate(df["Location"].unique()):
    temp_df = temp[temp["Location"] == loc]
    x, y = inx // 2, inx % 2
    print(loc, root_mean_squared_error(temp_df["true_cod"], temp_df["pred_cod"]))
    ax[x][y].scatter(temp_df["datetime"], temp_df["true_cod"], color = "green")
    ax[x][y].scatter(temp_df["datetime"], temp_df["pred_cod"], color = "red")
    ax[x][y].set_title(loc)

Try to increase number of estimator + adding validation based on good set of features for the model

In [ ]:
features_used = [f"cod_prev_{i}" for i in range(4, 13)] + \
                [f"temp_prev_{i}" for i in range(4, 13)] + \
                [f"ph_prev_{i}" for i in range(4, 13)] + \
                [f"tss_prev_{i}" for i in range(4, 13)] + \
                ["hour", "day", "month"]
X_train_arr, X_val_arr, X_test_arr, Y_train_arr, Y_val_arr, Y_test_arr = create_cv_split_with_val(df, features_used)

In [ ]:
best_params_xgboost_only_cod_with_val = optimize_xgboost(
    f"xgboost_study_only_cod_with_val_2025-05-04",
    f"xgboost_study_only_cod_with_val_2025-05-04",
    objective_xgboost_with_val
)
# slightly worse than model with only 200 trees

In [ ]:
best_params_catboost_only_cod_with_val = optimize_catboost(
    f"catboost_study_only_cod_with_val_{str(date.today())}",
    f"catboost_study_only_cod_with_val_{str(date.today())}",
    objective_catboost_with_val
) # not better than original model with 200 trees

Try to combine best 3 models

In [ ]:
def get_best_params_from_file(filename):
    study = optuna.create_study(
        study_name = filename,
        direction='minimize',
        storage = f"sqlite:///{filename}.db",
        sampler = RandomSampler(seed = 101),
        load_if_exists=True
    )
    return study.best_params

In [ ]:
# get dataframes for used
features_used = [f"cod_prev_{i}" for i in range(4, 13)] + \
                [f"temp_prev_{i}" for i in range(4, 13)] + \
                [f"ph_prev_{i}" for i in range(4, 13)] + \
                [f"tss_prev_{i}" for i in range(4, 13)] + \
                ["hour", "day", "month"]
X_train_arr, X_test_arr, Y_train_arr, Y_test_arr = create_cv_split(df, features_used)

# get best model params
best_params_xgboost = get_best_params_from_file("xgboost_study_2025-05-02")
params = {
    "n_estimators": 200,
    "verbosity": 0,
    "enable_categorical": True,
    "random_state": 101
}
for p in best_params_xgboost:
    params[p] = best_params_xgboost[p]

# train list of models
xgbr_arr = [XGBRegressor(**params)] * 5
Y_pred_xgboost_arr = []
for i in range(5):
    X_train, X_test = X_train_arr[i], X_test_arr[i]
    Y_train, Y_test = Y_train_arr[i], Y_test_arr[i]
    xgbr_arr[i].fit(X_train, Y_train)
    Y_pred_xgboost_arr.append(xgbr_arr[i].predict(X_test))

In [ ]:
# get dataframes for used
features_used = [f"cod_prev_{i}" for i in range(4, 13)] + \
                [f"temp_prev_{i}" for i in range(4, 13)] + \
                [f"ph_prev_{i}" for i in range(4, 13)] + \
                [f"tss_prev_{i}" for i in range(4, 13)] + \
                ["hour", "day", "month"]
X_train_arr, X_test_arr, Y_train_arr, Y_test_arr = create_cv_split(df, features_used)

# get best model params
best_params_lightgbm = get_best_params_from_file("lightgbm_study_2025-05-02")
params = {
    "n_estimators": 200,
    "verbosity": -1,
    "random_state": 101
}
for p in best_params_lightgbm:
    params[p] = best_params_lightgbm[p]

# train list of models
lgbr_arr = [LGBMRegressor(**params)] * 5
Y_pred_lightgbm_arr = []
for i in range(5):
    X_train, X_test = X_train_arr[i], X_test_arr[i]
    Y_train, Y_test = Y_train_arr[i], Y_test_arr[i]
    lgbr_arr[i].fit(X_train, Y_train)
    Y_pred_lightgbm_arr.append(lgbr_arr[i].predict(X_test))

In [ ]:
features_used = [f"cod_prev_{i}" for i in range(4, 13)] + \
                ["hour", "day", "month"] + \
                ["tss_prev_4", "tss_prev_5"]
X_train_arr, X_test_arr, Y_train_arr, Y_test_arr = create_cv_split(df, features_used)

# get best model params
best_params_catboost = get_best_params_from_file("catboost_study_best_comb_2025-05-03")
params = {
    "iterations": 200,
    "verbose": False,
    "random_seed": 37
}
for p in best_params_catboost:
    params[p] = best_params_catboost[p]

# train list of models
cbr_arr = [CatBoostRegressor(**params)] * 5
Y_pred_catboost_arr = []
for i in range(5):
    X_train, X_test = X_train_arr[i], X_test_arr[i]
    Y_train, Y_test = Y_train_arr[i], Y_test_arr[i]
    cbr_arr[i].fit(X_train, Y_train)
    Y_pred_catboost_arr.append(cbr_arr[i].predict(X_test))

In [ ]:
cv_rmse = 0
cv_rmse_xgboost = 0
cv_rmse_lightgbm = 0
cv_rmse_catboost = 0
for i in range(5):
    print(f"Error for xgboost at fold {i + 1}: {root_mean_squared_error(Y_test_arr[i], Y_pred_xgboost_arr[i])}")
    cv_rmse_xgboost += root_mean_squared_error(Y_test_arr[i], Y_pred_xgboost_arr[i])
    print(f"Error for lightgbm at fold {i + 1}: {root_mean_squared_error(Y_test_arr[i], Y_pred_lightgbm_arr[i])}")
    cv_rmse_lightgbm += root_mean_squared_error(Y_test_arr[i], Y_pred_lightgbm_arr[i])
    print(f"Error for catboost at fold {i + 1}: {root_mean_squared_error(Y_test_arr[i], Y_pred_catboost_arr[i])}")
    cv_rmse_catboost += root_mean_squared_error(Y_test_arr[i], Y_pred_catboost_arr[i])
    Y_pred = 0.05 * Y_pred_xgboost_arr[i] + 0.05 * Y_pred_lightgbm_arr[i] + 0.9 * Y_pred_catboost_arr[i]
    rmse = root_mean_squared_error(Y_test_arr[i], Y_pred)
    print(f"Error for combined model at fold {i + 1}: {rmse}")
    cv_rmse += rmse
print("CV error of simple model:")
print(f"xgboost: {cv_rmse_xgboost / 5}")
print(f"lightgbm: {cv_rmse_lightgbm / 5}")
print(f"catboost: {cv_rmse_catboost / 5}")
print("CV error of combined model")
print(cv_rmse / 5)
# peak at 2.770 (better than 2.7716 of catbooost with good features)